In [5]:
import cv2
import numpy as np
from collections import Counter
from decimal import Decimal, getcontext

# Increase the precision
getcontext().prec = 50

# Function to calculate the cumulative frequency
def calculate_cumulative_frequency(frequency):
    cumulative_freq = {}
    total = sum(frequency.values())
    cum_value = Decimal(0.0)

    for pixel, freq in sorted(frequency.items()):
        cumulative_freq[pixel] = (cum_value, cum_value + Decimal(freq) / Decimal(total))
        cum_value += Decimal(freq) / Decimal(total)

    return cumulative_freq

# Function to perform Arithmetic Encoding
def arithmetic_encoding(image, cumulative_freq):
    low = Decimal(0.0)
    high = Decimal(1.0)

    for pixel in image.flatten():
        range_ = high - low
        high = low + range_ * cumulative_freq[pixel][1]
        low = low + range_ * cumulative_freq[pixel][0]

    return (low + high) / 2  # The final code is any value between low and high

# Function to perform Arithmetic Decoding
def arithmetic_decoding(code, cumulative_freq, total_symbols):
    decoded_image = []
    sorted_freq = sorted(cumulative_freq.items(), key=lambda item: item[1][0])

    for _ in range(total_symbols):
        for pixel, (low, high) in sorted_freq:
            if low <= code < high:
                decoded_image.append(pixel)
                range_ = high - low
                code = (code - low) / range_
                break

    return np.array(decoded_image)

# Main function to apply Arithmetic Coding to an image
def main():
    # Load the image in grayscale mode
    image = cv2.imread('lenna.jpeg', cv2.IMREAD_GRAYSCALE)
    
    # Step 1: Calculate Frequency
    frequency = Counter(image.flatten())
    
    # Step 2: Calculate Cumulative Frequency
    cumulative_freq = calculate_cumulative_frequency(frequency)
    
    # Step 3: Encode Image
    encoded_value = arithmetic_encoding(image, cumulative_freq)
    
    # Step 4: Decode Image
    total_pixels = image.size
    decoded_image = arithmetic_decoding(encoded_value, cumulative_freq, total_pixels)
    decoded_image = decoded_image.reshape(image.shape)
    
    # Display the original and decoded images
    cv2.imshow('Original Image', image)
    cv2.imshow('Decoded Image', decoded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
